In [16]:
import sys
!{sys.executable} -m pip install psycopg2-binary

import psycopg2 as pg

In [17]:
conn = pg.connect(dbname="sequra", user="docker", password="docker", host="postgresql_sequra", port=5432)

## Load Mock Data into the database

In [18]:
with open("data-modeling-fixtures.sql", "r") as file:
    sql = file.read()

cursor = conn.cursor()

try:
    cursor.execute(sql)
    conn.commit()
except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()
    cursor.close()

cursor.close()

## Solution

In [19]:
sql = """

DROP TABLE IF EXISTS data_modeling.l1_cohort_months;

CREATE TABLE data_modeling.l1_cohort_months (
  month VARCHAR(50)
);

INSERT INTO data_modeling.l1_cohort_months VALUES 
  ('2017-01'),  ('2017-02'),  ('2017-03'),  ('2017-04'),  ('2017-05'),  ('2017-06'),  ('2017-07'),  ('2017-08'),  ('2017-09'),  ('2017-10'),  ('2017-11'),  ('2017-12'),
  ('2018-01'),  ('2018-02'),  ('2018-03'),  ('2018-04'),  ('2018-05'),  ('2018-06'),  ('2018-07'),  ('2018-08'),  ('2018-09'),  ('2018-10'),  ('2018-11'),  ('2018-12'),
  ('2019-01'),  ('2019-02'),  ('2019-03'),  ('2019-04'),  ('2019-05'),  ('2019-06'),  ('2019-07'),  ('2019-08'),  ('2019-09'),  ('2019-10'),  ('2019-11'),  ('2019-12'),
  ('2020-01'),  ('2020-02'),  ('2020-03'),  ('2020-04'),  ('2020-05'),  ('2020-06'),  ('2020-07'),  ('2020-08'),  ('2020-09'),  ('2020-10'),  ('2020-11'),  ('2020-12'),
  ('2021-01'),  ('2021-02'),  ('2021-03'),  ('2021-04'),  ('2021-05'),  ('2021-06'),  ('2021-07'),  ('2021-08'),  ('2021-09'),  ('2021-10'),  ('2021-11'),  ('2021-12'),
  ('2022-01'),  ('2022-02'),  ('2022-03'),  ('2022-04'),  ('2022-05'),  ('2022-06'),  ('2022-07'),  ('2022-08'),  ('2022-09'),  ('2022-10'),  ('2022-11'),  ('2022-12');

"""

cursor = conn.cursor()

try:
    cursor.execute(sql)
    conn.commit()
except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()
    cursor.close()

cursor.close()

In [20]:
sql = """

DROP TABLE IF EXISTS data_modeling.l1_cohort_month_periods;

CREATE TABLE data_modeling.l1_cohort_month_periods (
    period VARCHAR(50),
    min_days INTEGER,
    max_days INTEGER
);

INSERT INTO data_modeling.l1_cohort_month_periods VALUES
  ('M0',0,0),
  ('M1',1,30),
  ('M2',31,60),
  ('M3',61,90),
  ('M4',91,120),
  ('M5',121,150),
  ('M6',151,180),
  ('M7',181,210),
  ('M8',211,240),
  ('M9',241,270),
  ('M10',271,300),
  ('M11',301,330),
  ('M12',331,360);

"""

cursor = conn.cursor()

try:
    cursor.execute(sql)
    conn.commit()
except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()
    cursor.close()

cursor.close()

In [21]:
sql = """

WITH facts AS (
  SELECT 
      od.si_order_id,
      od.loan_reference,
      od.current_product_name,
      to_char(od.confirmed_at, 'YYYY-MM') AS cohort_month,
      od.record_date - od.confirmed_at AS cohort_month_period, 
      m.sector AS cohort_sector,
      m.name AS cohort_merchant,
      dt.default_state_name AS cohort_default_type
    FROM
      data_modeling.fact_orders_d od
    JOIN
      data_modeling.dim_default_type dt on od.default_type_id = dt.default_type_id
    JOIN
      data_modeling.dim_merchants m on od.merchant_id = m.merchant_id
),
aggregated_facts AS 
(
    SELECT
      l1cmp.period,
      l1cm.month,
      count(1) as fact_count
    FROM
      data_modeling.l1_cohort_months l1cm
    LEFT JOIN facts f ON
        f.cohort_month = l1cm.month   
    LEFT JOIN 
      data_modeling.l1_cohort_month_periods l1cmp ON 
        f.cohort_month_period >= l1cmp.min_days AND
        f.cohort_month_period <= l1cmp.max_days    
    GROUP BY 1, 2
)
  SELECT
    af.period,
    af.month,
    af.fact_count
  FROM aggregated_facts af
  WHERE 
    af.period IS NOT NULL

"""

cursor = conn.cursor('data_modeling_cursor')

try:
    
    cursor.itersize = 5000
    cursor.execute(sql)

    while True:
        rows = cursor.fetchmany(size=5000)

        for row in rows:
            print(row)
        
        if not rows:
            break

except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()

cursor.close()

In [22]:
conn.close()